In [3]:
from datetime import datetime
from dataclasses import dataclass

def hide_field(field:str) -> str:
    return '**민감정보 삭제**'

def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime('%Y-%m-%d %H:%M')

def show_original(event_field):
    return event_field

class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields

    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for(field, transformation) in self.serialization_fields.items()
        }

class Serialization:
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)

    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)
        
        event_class.serialize = serialize_method
        return event_class


@Serialization(
    username = show_original,
    password = hide_field,
    ip = show_original,
    timestamp = format_time
)
@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: str

l1 = LoginEvent('jaemin', '13123123','10.10.10.1', datetime(1993,1,1,1,10))
print(l1.serialize())

{'username': 'jaemin', 'password': '**민감정보 삭제**', 'ip': '10.10.10.1', 'timestamp': '1993-01-01 01:10'}


In [8]:
from functools import wraps
from types import MethodType

class DBDriver:
    def __init__(self, dbstring):
        self.dbstring = dbstring
    
    def execute(self, query):
        return f'{self.dbstring} 에서 쿼리 {query} 실행'

class inject_db_driver:
    def __init__(self, function):
        self.function = function
        wraps(self.function)(self)
    
    def __call__(self, dbstring):
        return self.function(DBDriver(dbstring))

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.__class__(MethodType(self.function, instance))

@inject_db_driver
def run_query(driver):
    return driver.execute('test function')

class DBHandler:
    @inject_db_driver
    def run_query(self, driver):
        return driver.execute(self.__class__.__name__)

run_query('test!!')
DBHandler().run_query('또 test!!!')

'또 test!!! 에서 쿼리 DBHandler 실행'